# Benchmark m4

How to fit several model and evaluate its predictions on three of subsets of M4 data. 


#### functools.partial

Return a new `partial` object which when called will behave like *func* called with the positional arguments args and keyword arguments keywords. 

- [Source](https://docs.python.org/2/library/functools.html)

## Methods  

Brief overview of the used methods

### `DeepAREstimator`

- [`gluonts.model.deepar`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

Construct a DeepAR estimator, implements an RNN-based model, close to the one described in

- Salinas, David, Valentin Flunkert, and Jan Gasthaus. “DeepAR: Probabilistic forecasting with autoregressive recurrent networks.” arXiv preprint arXiv:1704.04110 (2017).


### `MQCNNEstimator`

- [`gluonts.model.seq2seq`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.seq2seq.html)

An `MQDNNEstimator` with Convolutional Neural Network (CNN) as an encoder. Implements the MQ-CNN Forecaster, proposed in 

- Wen, Ruofeng, et al. “A multi-horizon quantile recurrent forecaster.” arXiv preprint arXiv:1711.11053 (2017).


### `SimpleFeedForwardEstimator`

- [`gluonts.model.simple_feedforward`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.simple_feedforward.html)

A simple multilayer perceptron model predicting the next target time-steps given the previous ones. 



In [1]:
# install gluonts (most recent version)
!pip install git+https://github.com/awslabs/gluon-ts.git

  Cloning https://github.com/awslabs/gluon-ts.git to c:\users\tm\appdata\local\temp\pip-req-build-_qy5pd3c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  Running command git clone -q https://github.com/awslabs/gluon-ts.git 'C:\Users\TM\AppData\Local\Temp\pip-req-build-_qy5pd3c'
  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\TM\Anaconda3\python.exe' 'C:\Users\TM\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' get_requires_for_build_wheel 'C:\Users\TM\AppData\Local\Temp\tmpyfhvh3gn'
       cwd: C:\Users\TM\AppData\Local\Temp\pip-req-build-_qy5pd3c
  Complete output (18 lines):
  Traceback (most recent call last):
    File "C:\Users\TM\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py", line 207, in <module>
      main()
    File "C:\Users\TM\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py", line 197, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
    File "C:\Users\TM\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py", line 54, in get_requires_for_build_wheel
      return hook(config_settings)
    File "C:\Users\TM\AppData\Local\Temp\pip-bu

In [2]:
# imports
import pandas as pd

import pprint
from functools import partial

# gluonts imports
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.trainer import Trainer

# gluonts models
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator


INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU


In [3]:
# gluonts version (important)
import gluonts
print(gluonts.__version__)

0.3.3


Number of series in m4 data: 

- Yearly: 23,000
- Quarterly: 24,000
- Monthly: 48,000
- Weekly: 359
- Daily: 4227
- Hourly: 414



In [4]:
datasets = [
    "m4_yearly",
]

print(datasets)

epochs = 1
num_batches_per_epoch = 50

['m4_yearly']


In [5]:
estimators = [
    partial(
        SimpleFeedForwardEstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
    partial(
        DeepAREstimator,
        distr_output=PiecewiseLinearOutput(8),
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
    partial(
        MQCNNEstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
]


INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU


In [6]:
def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict

In [7]:
if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            print(estimator)
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

functools.partial(<class 'gluonts.model.simple_feedforward._estimator.SimpleFeedForwardEstimator'>, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=1, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08))


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_yearly.


__init__() got an unexpected keyword argument 'use_feat_static_cat'
functools.partial(<class 'gluonts.model.deepar._estimator.DeepAREstimator'>, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=1, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08))


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_yearly.


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[23000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=20, freq="12M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=6, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=1, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True) on TrainDatasets(metadata=<MetaData freq='12M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='23000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=6>, train=<gluonts.dataset.common.FileDataset object at 0x000001F272B0EEB8>, test=<gluonts.datas

...

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 473457
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:02<00:00, 21.93it/s, avg_epoch_loss=495]
INFO:root:Epoch[0] Elapsed time 2.288 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=495.098392
INFO:root:Loading parameters from best epoch (0)
INFO:root:Final loss: 495.09839233398435 (occurred at epoch 0)
INFO:root:End model training
Running evaluation:   0%|          | 0/23000 [00:00<?, ?it/s]


Cannot extract prediction target since the index of forecast is outside the index of target
Index of forecast: DatetimeIndex(['1752-07-31', '1753-07-31', '1754-07-31', '1755-07-31',
               '1756-07-31', '1757-07-31'],
              dtype='datetime64[ns]', freq='12M')
 Index of target: DatetimeIndex(['1749-12-31', '1750-12-31', '1751-12-31', '1752-12-31',
               '1753-12-31', '1754-12-31', '1755-12-31', '1756-12-31',
               '1757-12-31', '1758-12-31', '1759-12-31', '1760-12-31',
               '1761-12-31', '1762-12-31', '1763-12-31', '1764-12-31',
               '1765-12-31', '1766-12-31', '1767-12-31', '1768-12-31',
               '1769-12-31', '1770-12-31', '1771-12-31', '1772-12-31',
               '1773-12-31', '1774-12-31', '1775-12-31', '1776-12-31',
               '1777-12-31', '1778-12-31', '1779-12-31', '1780-12-31',
               '1781-12-31', '1782-12-31', '1783-12-31', '1784-12-31',
               '1785-12-31'],
              dtype='datetime64[ns]',

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_yearly.


__init__() got an unexpected keyword argument 'use_feat_static_cat'


KeyError: "None of [Index(['dataset', 'estimator', 'RMSE', 'mean_wQuantileLoss', 'MASE', 'sMAPE',\n       'MSIS'],\n      dtype='object')] are in the [columns]"

In [ ]:
df[["MASE", "sMAPE", "dataset", "estimator"]]